# <div align=center > Lab3 : Circuit Simulation Engine </div>

###  <div> Names and IDs:

<div align=center > Ahmed Hussien Abdelbary 1180482 </br> Sandra Hany Helmy 1180563 </div></div>

In [1]:
import numpy as np

In [2]:
class Component:
    def __init__(self,componentType,node1,node2,value,intialValue):
        self.componentType = componentType
        self.node1 = node1
        self.node2 = node2
        self.value = value
        self.intialValue = intialValue
         
    def __str__(self):
        return '''
        
        Component Type: {componentType}
        Node 1: {node1}
        Node 2: {node2}
        value: {value}
        Intial Value: {intialValue}
        
        '''.format(componentType = self.componentType,
                   node1 = self.node1 , 
                   node2=self.node2 , 
                   value = self.value , 
                   intialValue = self.intialValue)
    
    
    def __repr__(self):
        return self.__str__()

In [3]:
class Circuit:
    def __init__(self,simTime,numberOfIterations,components):
        self.simTime = simTime
        self.numberOfIterations = numberOfIterations
        self.components = components
    
    def __str__(self):
        return '''
        Simulation Time: {simTime} 
        Number of Iterations: {numberOfIterations}
        Components: {components}
        '''.format(simTime = self.simTime , 
                   numberOfIterations = self.numberOfIterations , 
                   components = self.components)
    
    def getNumberOfNodes(self):
        nodeSet = set()
        for component in self.components:
            nodeSet.add(component.node1)
            nodeSet.add(component.node2)
        return len(nodeSet)
    
    def getNumberofIndependetVoltageSources(self):
        voltageSourcesCount = 0
        for component in self.components:
            if component.componentType == "Vsrc":
                voltageSourcesCount+=1
        return voltageSourcesCount
    
    def getVoltageSourcesNodes(self):
        VSnodes = np.zeros(shape = (self.getNumberofIndependetVoltageSources(),2), dtype = int)
        idx = 0
        for component in self.components:
            if component.componentType == "Vsrc":
                VSnodes[idx,0] = component.node1
                VSnodes[idx,1] = component.node2
                idx+=1
        return VSnodes
    
    def getCurrentSourcesNodes(self):
        CSnodes = np.zeros(shape = (self.getNumberofIndependetVoltageSources(),3), dtype = int)
        idx = 0
        for component in self.components:
            if component.componentType == "Isrc":
                CSnodes[idx,0] = component.node1
                CSnodes[idx,1] = component.node2
                CSnodes[idx,2] = component.value
                idx+=1
        return CSnodes

In [4]:
def createComponent(data):
    data = data.split()
    if len(data) < 5:
        return
    else:
        return Component(componentType = data[0],
                         node1 = int(data[1][1]),
                         node2 = int(data[2][1]),
                         value = float(data[3]),
                         intialValue = float(data[4]))


def readConfigFile(path):
    components = []
    circuit = Circuit(0,0,[])
    with open(path,'r') as file:
        for i,line in enumerate(file):
            if i== 0:
                circuit.simTime = line.strip()
            elif i == 1:
                circuit.numberOfIterations = line.strip()
            elif i > 1 :
                components.append(createComponent(line.strip()))
    components.pop(-1) #remove the last element flag
    circuit.components = components
    return circuit

In [5]:
circuits = []
for i in range(1,9):
    circuits.append(readConfigFile(path = "testcases/"+str(i)+".txt"))

In [6]:
'''
    The Simulation Engine Solves the following Equation
                       Ax=z 
    where A is a matrix Composed of 4 sub-matrices G,B,C,D
    z is a matrix composed of 2 sub-matrices i,e
    x is a matrix composed of 2 sub-matrices v,j
'''   

class SimulationEngine:
    def __init__(self,circuit):
        self.circuit = circuit
        self.VSCount = circuit.getNumberofIndependetVoltageSources()
        self.nodeCount  = circuit.getNumberOfNodes()
        self.numberOfIterations = circuit.numberOfIterations
        self.simulationTime = circuit.simTime
        
        
#----------------------------------------------Construct A sub-matrices------------------------------------------------
    
    def constructG(self):
        G = np.zeros(shape = (self.nodeCount,self.nodeCount) , dtype = float) 
        for component in self.circuit.components:
            if(component.componentType == 'R'):
                G[component.node1,component.node1] += 1/component.value
                G[component.node2,component.node2] += 1/component.value
                G[component.node1,component.node2] -= 1/component.value
                G[component.node2,component.node1] -= 1/component.value
        return G[1:,1:]
            
    def constructB(self):
        B = np.zeros(shape = (self.nodeCount,self.VSCount) , dtype = float)
        VSnodes = self.circuit.getVoltageSourcesNodes()
        for src_idx,source in enumerate(VSnodes):
            B[source[0],src_idx] = 1.0
            B[source[1],src_idx] = -1.0
        return B[1:]
    
    def constructC(self):
        C = self.constructB().T
        return C
    
    def constructD(self):
        D = np.zeros(shape = (self.VSCount,self.VSCount) , dtype = float)
        return D 
    
#                  -------------------------------Concat Matrix A---------------------------------
    def constructA(self):
        G = self.constructG()
        B = self.constructB()
        C = self.constructC()
        D = self.constructD()
        
        GB = np.concatenate((G,B),axis = 1)
        CD = np.concatenate((C,D),axis = 1)
        
        A = np.concatenate((GB,CD),axis = 0)
        
        return A
    
    
    
#-------------------------------------------------Construct Z sub-matrices---------------------------------------------

    def constructI(self):
        I = np.zeros( shape = (self.nodeCount,1) , dtype = float)
        CSnodes = self.circuit.getCurrentSourcesNodes()
        for src_idx,source in enumerate(CSnodes):
            I[source[0],0] = source[2] 
            I[source[1],0] = source[2] 
        return I[1:]
        
        
    def constructE(self):
        E = np.zeros( shape = (self.VSCount,1) , dtype = float)
        idx = 0
        for component in self.circuit.components:
            if(component.componentType == "Vsrc"):
                E[idx] = component.value
                idx+=1
        return E
    
    
#                   --------------------------------Concat Matrix z---------------------------------------
    def constructZ(self):
        I = self.constructI()
        E = self.constructE()
        
        Z = np.concatenate((I,E), axis = 0)
        
        return Z
    
    
    
#---------------------------------------Solve the linear System to find matrix X---------------------------------------
    def calculateX(self):
        A = self.constructA()
        Z = self.constructZ()
        
        X = np.round(np.linalg.inv(A) @ Z , 13)
        
        return X
    
#-----------------------------------------------print Solution---------------------------------------------------------
    
    def solve(self):
        X = np.squeeze(self.calculateX())
        
        for i in range(1,self.nodeCount):
            print("V"+str(i)+" = "+str(X[i-1]))
            
        for i in range(self.VSCount):
            print("I_Vsrc"+str(i)+" = "+str(X[i+self.nodeCount-1]))
        
        print("                     --------------------------------------------------------------------           \n")
        

In [7]:
# Part 1 Test Cases: First 4 Circuits e.g. 0:3

for circuit_idx,circuit in enumerate(circuits):
    if circuit_idx < 4:
        simulationEngine = SimulationEngine(circuit= circuit)
        print("Circuit: "+str(circuit_idx+1)+"\n")
        simulationEngine.solve()

Circuit: 1

V1 = 10.0
V2 = 5.0
I_Vsrc0 = -1.6666666666667
                     --------------------------------------------------------------------           

Circuit: 2

V1 = 10.0
V2 = 1.2
I_Vsrc0 = -0.044
                     --------------------------------------------------------------------           

Circuit: 3

V1 = -8.0
V2 = 24.0
V3 = 20.0
I_Vsrc0 = -0.4
I_Vsrc1 = 0.1
                     --------------------------------------------------------------------           

Circuit: 4

V1 = 2.0
V2 = 0.0
I_Vsrc0 = -4.0
                     --------------------------------------------------------------------           

